# Booking prices analysis

<br>

## Basic info:   
   * **author**: Slawomir Drzymala   
   * **code**: <a href="http://www.gooogle.com" target="_blank">github/sdrzymala</a>
   * **last update date**: 2019-10-27

## Description:
To achieve the given goal the following steps has been made:
   1. Get hotel/apartments/etc (property) details and prices for given city for the same length of stay for different days 
   1. Specify the office location and calculate the distance between the office and each property
   1. Analye the data and check the distribution of price per person per night
      * in total
      * excluding outliers (most luxurious properties)
      * for hotels only
      * for trusted properties only (with review score > 5)
      * for properties that are within the walking distance to the office
      * all together


# 1.1. Get config file and set up variables and params

In [1]:
# params

# read config from file
config_file_name = 'current_config.json'
#config_file_name = 'config_debug_local.json'

sql_query_search = "select * from dbo.search"
sql_query_search_result = "select * from dbo.search_result"
sql_query_city = "select * from dbo.city"
sql_query_destination = "select * from dbo.destination"
sql_query_distance_result = "select * from dbo.distance_result"

import json 

try:
    with open(config_file_name) as json_file:
        config = json.load(json_file)
except:
    raise ValueError("The config file was not found or file is corrupted")

db_connection_string = config["tech"]["db_connectionstring"]
price_per_person_per_night_limit = config["analysis"]["price_per_person_per_night_limit"]
walking_distance_limit_meters = config["analysis"]["walking_distance_limit_meters"]
gmaps_api_key = config["tech"]["google_api_key"]



# 1.2. Import libraries

In [2]:
# import
import pyodbc
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import math

%matplotlib inline
import matplotlib.pyplot as plt

# 1.3 Import data

In [3]:
# load data from database
# connect
cnxn = pyodbc.connect(db_connection_string)

# load all tables

df_search_raw = pd.read_sql(sql_query_search,cnxn)
df_search_result_raw = pd.read_sql(sql_query_search_result,cnxn)
df_city_raw = pd.read_sql(sql_query_city,cnxn)
df_destination_raw = pd.read_sql(sql_query_destination,cnxn)
df_distance_result_raw = pd.read_sql(sql_query_distance_result,cnxn)


In [4]:
# preview original data
display(df_search_raw.head())
display(df_search_result_raw.head())
display(df_city_raw.head())
display(df_destination_raw.head())
display(df_distance_result_raw.head())


# 1.4 Prepare data frames

## 1.4.1 Prepare data frame: search

In [5]:
# copy original data
df_search = df_search_raw

# join search df with city df to retrive the name of the city and the country
df_search = pd.merge(df_search_raw, df_city_raw, on='city_id', how='inner')

# drop not needed columns
df_search_columns_to_drop = ["inserted_at",
                            "city_id"]
df_search.drop(df_search_columns_to_drop, inplace=True, axis=1)



# assign to the final data frame
df_search.head()

## 1.4.2 Prepare data frame: search result

In [6]:
# copy original data
df_search_result = df_search_result_raw



# join search_result df with city df to retrive the name of the city and the country
df_search_result = pd.merge(df_search_result, df_city_raw, on='city_id', how='inner')



# join search_result df with search to retirve the information about the length stay
df_search_result = pd.merge(df_search_result, df_search_raw, on='search_id', how='inner')



# handle null values
df_search_result["property_type_badge"].fillna("N/A", inplace = True) 
df_search_result["no_reviews"].fillna(0, inplace = True) 
df_search_result["review_score"].fillna(0, inplace = True) 



# add new (calculated) attributes
df_search_result['property_type'] = np.select(
                                                    [
                                                        (df_search_result['hotel_rating_star'].notnull() | df_search_result['property_name'].str.contains("hotel", case=False)),
                                                        df_search_result['property_name'].str.contains("hostel", case=False),
                                                        df_search_result['property_type_badge'].str.contains("condo", case=False),
                                                        df_search_result['property_type_badge'].str.contains("villa", case=False),
                                                        df_search_result['proposed_room_type'].str.contains("apartment", case=False) | df_search_result['property_type_badge'].str.contains("apartment", case=False) | df_search_result['property_name'].str.contains("apartment", case=False) | df_search_result['property_name'].str.contains("apartament", case=False)
                                                        
                                                    ], 
                                                    [
                                                            'Hotel',
                                                            'Hostel',
                                                            'Condo Hotel',
                                                            'Villa',
                                                            'Apartment'
                                                    ], 
                                                    default='Unknown'
                                                )
df_search_result['is_hotel'] = np.where(df_search_result['property_type'] == 'Hotel', True, False)
df_search_result['review_score_int'] = df_search_result['review_score'].round().astype(int) 
df_search_result['review_score_bin'] = pd.cut(df_search_result['review_score_int'], [float("-inf"), 0, 2, 4, 6, 8, 10,np.inf], include_lowest=True).astype(str)
df_search_result['price_per_person_per_night'] = df_search_result['promo_price']/df_search_result['no_nights']/df_search_result['no_adults']
df_search_result['is_price_per_person_per_night_above_limit'] = np.where(df_search_result['price_per_person_per_night'] > price_per_person_per_night_limit, True, False)
df_search_result['total_price_price_outlier_zscore'] = np.abs(stats.zscore(df_search_result['promo_price'])) 
df_search_result['is_total_price_price_outlier_zscore'] = np.where(df_search_result['total_price_price_outlier_zscore'] > 3, True, False)
df_search_result['booking_dates'] = df_search_result['check_in_date'].apply(lambda x: x.strftime('%Y-%m-%d')) + " to " + df_search_result['check_out_date'].apply(lambda x: x.strftime('%Y-%m-%d')) 



# drop not needed columns
df_search_result_columns_to_drop = ["property_offer_url",
                                    "page_no",
                                    "original_price",
                                    "inserted_at_y",
                                    "get_only_avaliable",
                                    "inserted_at_x",
                                    "city_id_y",
                                    "search_starttime",
                                    "search_endtime",
                                    "search_date",
                                    "currency",
                                    "language",
                                    "no_nights",
                                    "no_adults",
                                    "no_childrens",
                                    "no_rooms",
                                    "is_business_trip",
                                    "get_only_avaliable"
                                   ]
df_search_result.drop(df_search_result_columns_to_drop, inplace=True, axis=1)



# rename columns
df_search_result = df_search_result.rename(columns={
                          'city_id_x': 'city_id'
                        , 'promo_price': 'total_price'
                        })


# review
display(df_search_result_raw.head())
display(df_search_result.head())

In [7]:
z_score_outlier_limit_1 = 1
z_score_outlier_limit_2 = 2
z_score_outlier_limit_3 = 3

df_search_result_1 = df_search_result[df_search_result['total_price_price_outlier_zscore'] < z_score_outlier_limit_1]
df_search_result_2 = df_search_result[df_search_result['total_price_price_outlier_zscore'] < z_score_outlier_limit_2]
df_search_result_3 = df_search_result[df_search_result['total_price_price_outlier_zscore'] < z_score_outlier_limit_3] 


n_standard = len(df_search_result)
n_1 = len(df_search_result_1)
n_2 = len(df_search_result_2)
n_3 = len(df_search_result_3)


min_standard = int(df_search_result["total_price"].min())
min_1 = int(df_search_result_1["total_price"].min())
min_2 = int(df_search_result_2["total_price"].min())
min_3 = int(df_search_result_3["total_price"].min())

mean_standard = int(df_search_result["total_price"].mean())
mean_1 = int(df_search_result_1["total_price"].mean())
mean_2 = int(df_search_result_2["total_price"].mean())
mean_3 = int(df_search_result_3["total_price"].mean())

max_standard = int(df_search_result["total_price"].max())
max_1 = int(df_search_result_1["total_price"].max())
max_2 = int(df_search_result_2["total_price"].max())
max_3 = int(df_search_result_3["total_price"].max())

std_standard = int(df_search_result["total_price"].std())
std_1 = int(df_search_result_1["total_price"].std())
std_2 = int(df_search_result_2["total_price"].std())
std_3 = int(df_search_result_3["total_price"].std())

median_standard = int(df_search_result["total_price"].median())
median_1 = int(df_search_result_1["total_price"].median())
median_2 = int(df_search_result_2["total_price"].median())
median_3 = int(df_search_result_3["total_price"].median())

fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6), (ax7, ax8)) = plt.subplots(4, 2, figsize=(15, 25))

sns.distplot(df_search_result['total_price'], ax=ax1).set_title("Standard, n: {0}, percent of population: {1}%, min: {2}, mean: {3}, median: {4}, max: {5}, stdev: {6}".format(n_standard, int(n_standard/n_standard*100), min_standard, mean_standard, median_standard, max_standard, std_standard))
sns.boxplot(x=df_search_result['total_price'], ax=ax2)

sns.distplot(df_search_result_3['total_price'], ax=ax3).set_title("Z-Score 3, n: {0}, percent of population: {1}%, min: {2}, mean: {3}, median: {4}, max: {5}, stdev: {6}".format(n_3, int(n_3/n_standard*100), min_3, mean_3, median_3, max_3, std_3))
sns.boxplot(x=df_search_result_3['total_price'], ax=ax4)

sns.distplot(df_search_result_2['total_price'], ax=ax5).set_title("Z-Score 2, n: {0}, percent of population: {1}%, min: {2}, mean: {3}, median: {4}, max: {5}, stdev: {6}".format(n_2, int(n_2/n_standard*100), min_2, mean_2, median_2, max_2, std_2))
sns.boxplot(x=df_search_result_2['total_price'], ax=ax6)

sns.distplot(df_search_result_1['total_price'], ax=ax7).set_title("Z-Score 1, n: {0}, percent of population: {1}%, min: {2}, mean: {3}, median: {4}, max: {5}, stdev: {6}".format(n_1, int(n_1/n_standard*100), min_1, mean_1, median_1, max_1, std_1))
sns.boxplot(x=df_search_result_1['total_price'], ax=ax8)

plt.show()


## 1.4.3 Prepare data frame: distance result

In [8]:
# copy original data
df_distance_result = df_distance_result_raw



# join distiance_result df with city df to retrive the name of the city and the country
df_distance_result = pd.merge(df_distance_result, df_city_raw, on='city_id', how='inner')



# join search_result df with city df to retrive the name of the city and the country
df_distance_result = pd.merge(df_distance_result, df_destination_raw, on='destination_id', how='inner')



# join to the search results booking properties to get the location of the property
distinct_booking_properties_coordinates = df_search_result_raw[['booking_property_id','property_name','coordinates_long', 'coordinates_lat']].drop_duplicates()
df_distance_result = pd.merge(df_distance_result, distinct_booking_properties_coordinates, on='booking_property_id', how='inner')

# get average price of given property and join to distance result
property_mean_prices = df_search_result \
        .groupby('booking_property_id') \
        .agg({
                'price_per_person_per_night': ['mean']
        }) \
        .reset_index()
property_mean_prices.columns = ['booking_property_id', 'price_per_person_per_night_mean']
property_mean_prices['price_per_person_per_night_mean'] = round(property_mean_prices['price_per_person_per_night_mean'],0)
df_distance_result = pd.merge(df_distance_result, property_mean_prices, on='booking_property_id', how='inner')


# add new attributes
df_distance_result['walking_duration_minutes'] = (df_distance_result['walking_duration_seconds']/60).round().astype(int)
df_distance_result['walking_duration_minutes_bin'] = pd.cut(df_distance_result['walking_duration_minutes'], [0, 15, 30, 45, 60, 75, 90, 105, 120, np.inf], include_lowest=True).astype(str)
df_distance_result['driving_duration_minutes'] = (df_distance_result['driving_duration_seconds']/60).round().astype(int)
df_distance_result['driving_duration_minutes_bin'] = pd.cut(df_distance_result['driving_duration_minutes'], [0, 15, 30, 45, 60, 75, 90, 105, 120, np.inf], include_lowest=True).astype(str)
df_distance_result['walking_distance_km'] = (df_distance_result['walking_distance_meters']/1000).round().astype(int)
df_distance_result['walking_distance_km_bin'] = pd.cut(df_distance_result['walking_distance_km'], [0, 2, 4, 6, 8, 10, np.inf], include_lowest=True).astype(str)
df_distance_result['driving_distance_km'] = (df_distance_result['driving_distance_meters']/1000).round().astype(int)
df_distance_result['driving_distance_km_bin'] = pd.cut(df_distance_result['driving_distance_km'], [0, 2, 4, 6, 8, 10, np.inf], include_lowest=True).astype(str)
df_distance_result['is_within_walking_distance_limit'] = np.where(df_distance_result['walking_distance_meters'] <= walking_distance_limit_meters, True, False)



# drop not needed columns
df_distance_result_columns_to_drop = ["transit_duration_seconds",
                                      "transit_distance_meters",
                                      "transit_reponse_status",
                                      "city_id_y"
                                   ]
df_distance_result.drop(df_distance_result_columns_to_drop, inplace=True, axis=1)



# rename columns
df_distance_result = df_distance_result.rename(columns={
                          'city_id_x': 'city_id',
                          'promo_price': 'total_price',
                          'coordinates_long_x': 'destination_coordinates_long',
                          'coordinates_lat_x': 'destination_coordinates_lat',
                          'coordinates_long_y': 'property_coordinates_long',
                          'coordinates_lat_y': 'property_coordinates_lat'
                        })



display(df_distance_result_raw.head())
display(df_distance_result.head())

## 1.5 Prepare final data set with all informations

In [9]:
df_distance_result_x = df_distance_result[df_distance_result['destination_id'] == 1] 
df_distance_result_x.columns = [str(col) + '_x' for col in df_distance_result_x.columns]
df_distance_result_x = df_distance_result_x.rename(columns={'booking_property_id_x': 'booking_property_id'})


df_search_result = pd.merge(df_search_result, df_distance_result_x, on='booking_property_id', how='inner')


## 1.6 Temporary testing and verification (optional)

In [10]:
# test show bins stats
#plt.figure(figsize=(16, 6))
#sns.countplot(x = 'walking_distance_km_bin',
#              data = df_distance_result,
#              order = df_distance_result['walking_distance_km_bin'].value_counts().index)
#plt.show()



# test show all data
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#    display(df_distance_result[0:1000])

## 1.7 Return data

In [11]:
# share data
%store df_distance_result
%store df_search_result
%store df_search
%store price_per_person_per_night_limit
%store walking_distance_limit_meters
%store gmaps_api_key